# Solution: Using OpenAI's Web Search Tool

In [2]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, WebSearchTool, function_tool, trace

dotenv.load_dotenv()

True

In [3]:
chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [4]:
# This is the same code as in the rag.ipynb notebook


@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

## The Trick

Here comes the trick: Instead of creating an Exa Search MCP, we are simply passing the `WebSearchTool` as a tool to the agent 

In [7]:
calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:
        0) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        1) If you couldn't find the exact match for the food or you need to look up the ingredients, search the web to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie
        information of the ingredients to make sure the information you provide is consistent.
        2) Then, if it's about a meal, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use web search to find the exact recipe and ingredients.
    * Once you know the ingredients, always use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 2 times.
    """,
    tools=[calorie_lookup_tool, WebSearchTool()],
)

In [8]:
with trace("Nutrition Assistant with Web Search"):
    result = await Runner.run(
        calorie_agent_with_search, "How many calories are in an english breakfast?"
    )
    print(result.final_output)

A full English breakfast typically ranges from about 600 to 1,600 kcal per serving, depending on portions and cooking method. Common estimates fall around 800–1,200 kcal. ([bbcgoodfoodme.com](https://www.bbcgoodfoodme.com/recipes/the-ultimate-makeover-full-english-breakfast/?utm_source=openai))
